# PyTorch ONNX Exporter new features and architecture

Jan 2026

![infographics](<pt-onnx-infographics.png>)

## `dynamo=True` is the default

- The New Default: Starting from PyTorch 2.9, the **`dynamo=True`** option is the **default and recommended** way to export models to ONNX.
- Core Shift: It moves away from the older TorchScript-based capture mechanism to a torch.export based modern stack.
- Deprecation Plan: While the TorchScript exporter (dynamo=False) is currently usable, it is planned for eventual deprecation in alignment with PyTorch core's handling of TorchScript.

## New options in `export()`

```py
torch.onnx.export(
    model, args, kwargs=kwargs,
    # New way of expressing dynamic shapes (more examples later)
    dynamic_shapes=({0: "batch", 1: "sequence_len"}),
    # dynamic_axes=...,  # Deprecated
    dynamo=True,  # Default (2.9)
    report=True,  # Creates a markdown report
    verify=True,  # Runs onnx runtime on the example
    optimize=True, # Runs onnxscript graph optimizations
) -> torch.onnx.ONNXProgram
```

## What happens inside `torch.onnx.export`

torch.export() **captures FX** graph
-> **translate** and build ONNX IR
-> graph **optimization** with ONNX Script

Entry point is at: https://github.com/pytorch/pytorch/blob/0ad306cac740eaf2ce582e2bdf097cc61d929a40/torch/onnx/_internal/exporter/_core.py#L1282

![diagram](https://raw.githubusercontent.com/justinchuby/diagrams/refs/heads/main/pytorch/torch-export-flow.svg)

In [ ]:
## FX graph and the ExportedProgram

In [11]:
import torch
import torch.export

class Mod(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.randn(10, 10))

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        a = torch.sin(x)
        a.add_(y)
        return a * self.weight

example_args = (torch.randn(10, 10), torch.randn(10, 10))

# Important to set to eval mode before exporting
mod = Mod().eval()
exported_program: "ExportedProgram" = torch.export.export(mod, args=example_args)
print(exported_program)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_weight: "f32[10, 10]", x: "f32[10, 10]", y: "f32[10, 10]"):
             # File: /tmp/ipykernel_246584/1311693341.py:10 in forward, code: a = torch.sin(x)
            sin: "f32[10, 10]" = torch.ops.aten.sin.default(x);  x = None
            
             # File: /tmp/ipykernel_246584/1311693341.py:11 in forward, code: a.add_(y)
            add_: "f32[10, 10]" = torch.ops.aten.add_.Tensor(sin, y);  sin = y = None
            
             # File: /tmp/ipykernel_246584/1311693341.py:12 in forward, code: return a * self.weight
            mul: "f32[10, 10]" = torch.ops.aten.mul.Tensor(add_, p_weight);  add_ = p_weight = None
            return (mul,)
            
Graph signature: 
    # inputs
    p_weight: PARAMETER target='weight'
    x: USER_INPUT
    y: USER_INPUT
    
    # outputs
    mul: USER_OUTPUT
    
Range constraints: {}



In [12]:
decomposed = exported_program.run_decompositions()
print(decomposed)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_weight: "f32[10, 10]", x: "f32[10, 10]", y: "f32[10, 10]"):
             # File: /tmp/ipykernel_246584/1311693341.py:10 in forward, code: a = torch.sin(x)
            sin: "f32[10, 10]" = torch.ops.aten.sin.default(x);  x = None
            
             # File: /tmp/ipykernel_246584/1311693341.py:11 in forward, code: a.add_(y)
            add: "f32[10, 10]" = torch.ops.aten.add.Tensor(sin, y);  sin = y = None
            
             # File: /tmp/ipykernel_246584/1311693341.py:12 in forward, code: return a * self.weight
            mul: "f32[10, 10]" = torch.ops.aten.mul.Tensor(add, p_weight);  add = p_weight = None
            return (mul,)
            
Graph signature: 
    # inputs
    p_weight: PARAMETER target='weight'
    x: USER_INPUT
    y: USER_INPUT
    
    # outputs
    mul: USER_OUTPUT
    
Range constraints: {}



## Translation to ONNX

In [13]:
onnx_program = torch.onnx.export(exported_program, verify=True, report=True)
print(onnx_program)

[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
x input_kind: InputKind.USER_INPUT persistent: None
y input_kind: InputKind.USER_INPUT persistent: None
p_weight input_kind: InputKind.PARAMETER persistent: None
[torch.onnx] Translate the graph into ONNX... ✅
[torch.onnx] Check the ONNX model...
[torch.onnx] Check the ONNX model... ✅
[torch.onnx] Execute the model with ONNX Runtime...
[torch.onnx] Execute the model with ONNX Runtime... ✅
[torch.onnx] Verify output accuracy...
[torch.onnx] Verify output accuracy... ✅
[torch.onnx] Export report has been saved to 'onnx_export_2026-01-05_12-23-00-069243_success.md'.
ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.10.0.dev20251028+cpu',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            in

In [14]:
onnx_program.save("model.onnx")

In [7]:
!onnxvis model.onnx

Loading extensions...
I0000 00:00:1767644264.603678  247352 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Loaded 9 adapters:
 - TFLite adapter (Flatbuffer)
 - TFLite adapter (MLIR)
 - TF adapter (MLIR)
 - TF adapter (direct)
 - GraphDef adapter
 - Pytorch adapter (exported program)
 - MLIR adapter
 - ONNX adapter
 - JSON adapter

Starting Model Explorer server at:
http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model.onnx%22%7D%5D%7D

Press Ctrl+C to stop.
gio: http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/home/justinchu/dev/talk-torch-onnx-apis-architecture/src/model.onnx%22%7D%5D%7D: Operation not supported
Stopping server...
^C


## Model in `onnx_program.model` is an onnx_ir.Model

- You can run any ONNX->ONNX transformation on it.
- The exporter by default runs ONNX Script pattern replacement and whole graph optimization. These are robust, in-memory graph passes the team has created
- Low memory consumption by sharing tensor data with the PyTorch model

In [ ]:
# Explore the IR model

model = onnx_program.model
print("Model has", len(model.graph), "nodes")

print("All initializers:")
for init in model.graph.initializers.values():
    print(" ", init)

Model has 3 nodes
All initializers:
  %"weight"<FLOAT,[10,10]>{TorchTensor(...)}


In [17]:
print(model.graph.initializers["weight"].const_value.raw is mod.weight)

True


In [18]:
model.graph.initializers["weight"].const_value.display()

TorchTensor<FLOAT,[10,10]>(Parameter containing: tensor([[-1.0907e+00, -1.5376e+00,  8.3582e-01,  3.7708e-01, 
-1.9148e+00, -1.8229e-03, -1.1517e+00, -1.6128e+00,  1.1214e+00,  7.3366e-01], [ 2.7739e-01, -1.5673e-01, 
-2.2370e+00,  1.3048e-01, -1.7112e+00, 3.5006e-01, -3.7150e-01, -9.0543e-01, -7.0478e-01,  1.1714e+00], 
[-2.7062e-01,  1.1843e+00,  6.0908e-01,  2.0033e-01, -1.4659e+00, 1.8297e+00,  4.3489e-01,  1.1888e+00,  
1.2808e+00,  1.8380e+00], [ 7.8664e-01, -4.6764e-01, -8.6129e-02,  1.0419e+00, -1.8287e+00, -2.8274e-01, 
-1.8635e+00, -6.5279e-01, -1.1301e+00, -9.4622e-01], [-1.9503e-01,  1.1259e+00, -5.4191e-01, -1.1867e+00, 
-3.9510e-01, -1.0607e+00, -9.9146e-01,  2.3685e-01,  1.0492e+00, -2.1548e-02], [ 4.1088e-01,  5.8937e-01, 
-5.9615e-01, -4.8070e-01,  7.8016e-01, 1.4684e+00, -1.7752e-01, -6.9450e-01,  1.9617e-01,  9.8532e-01], 
[-5.0937e-01, -1.3652e-02,  1.8041e+00, -2.4031e+00,  1.9256e+00, -1.7330e+00, -9.1820e-01, -5.3262e-01, 
-3.4171e-01, -5.4438e-01], [ 1.8903e-01,  5.2425e-01, -6.6874e-01,  9.3434e-01,  4.6576e-01, 1.1994e-01, 
-9.7236e-01,  3.6537e-01,  7.6553e-01, -1.4067e-01], [-1.2247e-01, -1.8754e-01, -5.6873e-01, -1.3509e+00, 
-4.4623e-01, -1.1999e-01,  7.1742e-01, -2.9808e-01, -5.4438e-01,  2.0525e+00], [ 1.4098e+00,  3.8164e-01,  
1.5952e-01,  7.4772e-01,  3.3845e-01, 9.4168e-01, -1.1702e-01,  6.0295e-01, -1.0441e+00,  3.4880e-01]], 
requires_grad=True), name='weight')

Min: -2.4031074047088623, Max: 2.0525074005126953, NaN count: 0, Inf count: 0
Sparsity (abs<1e-06): 0.00
Histogram:
       6 ┼                                                ╭╮
       5 ┤                                                ││
       4 ┤                                                ││
       4 ┤                                ╭╮     ╭╮    ╭╮ ││     ╭╮
       3 ┤                         ╭╮     │╰╮   ╭╯│    ││ ││     ││      ╭╮
       2 ┤           ╭╮        ╭╮╭─╯│  ╭──╯ │╭──╯ ╰─╮ ╭╯│ │╰╮ ╭╮ │╰╮ ╭─╮╭╯│         ╭╮
       2 ┤           ││        │││  │  │    ││      │ │ │ │ │ ││ │ │ │ ││ │         ││
       1 ┼╮╭╮    ╭──╮││╭──╮╭╮ ╭╯╰╯  │  │    ╰╯      ╰╮│ ╰─╯ ╰─╯╰╮│ ╰─╯ ││ │╭╮╭─╮    │╰─╮╭
       0 ┤╰╯╰────╯  ╰╯╰╯  ╰╯╰─╯     ╰──╯             ╰╯         ╰╯     ╰╯ ╰╯╰╯ ╰────╯  ╰╯
    -2.4031  -1.9575  -1.5120  -1.0107  -0.5652  -0.0639  0.3817  0.8272  1.2171  1.6626

In [19]:
print("All users of the initializer:", model.graph.initializers["weight"].uses())

All users of the initializer: (Usage(node=Node(name='node_mul', domain='', op_type='Mul', inputs=(SymbolicTensor(name='add', type=Tensor(FLOAT), shape=Shape([10, 10]), producer='node_add', index=0), SymbolicTensor(name='weight', type=Tensor(FLOAT), shape=Shape([10, 10]), const_value={TorchTensor(...)})), attributes={}, overload='', outputs=(SymbolicTensor(name='mul', type=Tensor(FLOAT), shape=Shape([10, 10]), producer='node_mul', index=0),), version=20, doc_string=None), idx=1),)


## Multiple ways to represent dynamic shapes

